In [1]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import pandas as pd
from datetime import datetime
from pathlib import Path


def convert_mongodb_fields(obj):
    if isinstance(obj, dict):
        if "$oid" in obj:
            return str(obj["$oid"])
        elif "$date" in obj:
            return datetime.fromtimestamp(obj["$date"] / 1000)
        else:
            return {k: convert_mongodb_fields(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_mongodb_fields(item) for item in obj]
    else:
        return obj


users_path = Path(r'C:\Users\gandh\OneDrive\Documents\JSON\users.json')
receipts_path = Path(r'C:\Users\gandh\OneDrive\Documents\JSON\receipts.json')
brands_path = Path(r'C:\Users\gandh\OneDrive\Documents\JSON\brands.json')


users_data = []
with open(users_path, 'r') as f:
    for line in f:
        try:
            json_obj = json.loads(line)
            processed_obj = convert_mongodb_fields(json_obj)
            users_data.append(processed_obj)
        except json.JSONDecodeError:
            print(f"Skipping malformed line in users.json: {line}")

users = pd.DataFrame(users_data)


receipts_data = []
with open(receipts_path, 'r') as f:
    for line in f:
        try:
            json_obj = json.loads(line)
            processed_obj = convert_mongodb_fields(json_obj)
            receipts_data.append(processed_obj)
        except json.JSONDecodeError:
            print(f"Skipping malformed line in receipts.json: {line}")

receipts = pd.DataFrame(receipts_data)

brands_data = []
with open(brands_path, 'r') as f:
    for line in f:
        try:
            json_obj = json.loads(line)
            processed_obj = convert_mongodb_fields(json_obj)
            brands_data.append(processed_obj)
        except json.JSONDecodeError:
            print(f"Skipping malformed line in brands.json: {line}")

brands = pd.DataFrame(brands_data)

users.to_csv(r'C:\Users\gandh\OneDrive\Documents\JSON\cleaned_users.csv', index=False)
receipts.to_csv(r'C:\Users\gandh\OneDrive\Documents\JSON\cleaned_receipts.csv', index=False)
brands.to_csv(r'C:\Users\gandh\OneDrive\Documents\JSON\cleaned_brands.csv', index=False)

print("Users Data:")
print(users.head())

print("\nReceipts Data:")
print(receipts.head())

print("\nBrands Data:")
print(brands.head())

Users Data:
                        _id  active             createdDate  \
0  5ff1e194b6a9d73a3a9f1052    True 2021-01-03 10:24:04.800   
1  5ff1e194b6a9d73a3a9f1052    True 2021-01-03 10:24:04.800   
2  5ff1e194b6a9d73a3a9f1052    True 2021-01-03 10:24:04.800   
3  5ff1e1eacfcf6c399c274ae6    True 2021-01-03 10:25:30.554   
4  5ff1e194b6a9d73a3a9f1052    True 2021-01-03 10:24:04.800   

                lastLogin      role signUpSource state  
0 2021-01-03 10:25:37.858  consumer        Email    WI  
1 2021-01-03 10:25:37.858  consumer        Email    WI  
2 2021-01-03 10:25:37.858  consumer        Email    WI  
3 2021-01-03 10:25:30.597  consumer        Email    WI  
4 2021-01-03 10:25:37.858  consumer        Email    WI  

Receipts Data:
                        _id  bonusPointsEarned  \
0  5ff1e1eb0a720f0523000575              500.0   
1  5ff1e1bb0a720f052300056b              150.0   
2  5ff1e1f10a720f052300057a                5.0   
3  5ff1e1ee0a7214ada100056f                5.0   
4

In [3]:
# Check for missing values
print("Missing values in Users:")
print(users.isnull().sum())

print("\nMissing values in Receipts:")
print(receipts.isnull().sum())

print("\nMissing values in Brands:")
print(brands.isnull().sum())

Missing values in Users:
_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

Missing values in Receipts:
_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

Missing values in Brands:
_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64


In [4]:
# Check for duplicates in Users
print("Duplicates in Users:", users.duplicated().sum())

# Check for duplicates in Receipts (excluding unhashable columns)
# Use 'object' for string columns and 'datetime64' for datetime columns
receipts_hashable = receipts.select_dtypes(include=['int', 'float', 'object', 'bool', 'datetime64'])
print("Duplicates in Receipts:", receipts_hashable.duplicated().sum())

# Check for duplicates in Brands
print("Duplicates in Brands:", brands.duplicated().sum())

Duplicates in Users: 283


TypeError: unhashable type: 'list'

In [5]:
# Check if all userIds in Receipts exist in Users
invalid_user_ids = receipts[~receipts['userId'].isin(users['_id'])]
print("Invalid userIds in Receipts:", invalid_user_ids.shape[0])

# Check if all barcodes in ReceiptItems exist in Brands
receipt_items = pd.json_normalize(receipts['rewardsReceiptItemList'].explode())
invalid_barcodes = receipt_items[~receipt_items['barcode'].isin(brands['barcode'])]
print("Invalid barcodes in ReceiptItems:", invalid_barcodes.shape[0])

Invalid userIds in Receipts: 148
Invalid barcodes in ReceiptItems: 7299


In [6]:
# Check for outliers in totalSpent (Receipts)
print("Summary statistics for totalSpent in Receipts:")
print(receipts['totalSpent'].describe())

# Check for outliers in purchaseDate (Receipts)
print("\nEarliest and latest purchase dates in Receipts:")
print("Earliest:", receipts['purchaseDate'].min())
print("Latest:", receipts['purchaseDate'].max())

Summary statistics for totalSpent in Receipts:
count      684
unique      94
top       1.00
freq       172
Name: totalSpent, dtype: object

Earliest and latest purchase dates in Receipts:
Earliest: 2017-10-29 20:00:00
Latest: 2021-03-08 12:37:13


In [ ]:
# Data Quality Issues

1. Missing Values
- Users:
  - `lastLogin`: 62 missing values.
  - `signUpSource`: 48 missing values.
  - `state`: 56 missing values.
- Receipts:
  - `bonusPointsEarned`: 575 missing values.
  - `bonusPointsEarnedReason`: 575 missing values.
  - `finishedDate`: 551 missing values.
  - `pointsAwardedDate`: 582 missing values.
  - `pointsEarned`: 510 missing values.
  - `purchaseDate`: 448 missing values.
  - `purchasedItemCount`: 484 missing values.
  - `rewardsReceiptItemList`: 440 missing values.
  - `totalSpent`: 435 missing values.
- Brands:
  - `category`: 155 missing values.
  - `categoryCode`: 650 missing values.
  - `topBrand`: 612 missing values.
  - `brandCode`: 234 missing values.

2. Duplicate Records
- **Users**: 283 duplicate records.

3. Invalid Relationships
- **Receipts**: 148 invalid `userId` values.
- **ReceiptItems**: 7299 invalid `barcode` values.

4. Outliers or Anomalies
- **Receipts**: Outliers in the `totalSpent` column (e.g., a maximum value of 1000).

5. Unhashable Columns
- **Receipts**: The `rewardsReceiptItemList` column contains unhashable lists.

## Recommendations
1. Investigate and fix missing values.
2. Remove duplicates and implement unique constraints.
3. Ensure referential integrity by validating foreign keys.
4. Investigate and address outliers.
5. Convert unhashable columns to hashable types if needed.